# Twitter Disaster Detection Model 

In [1]:

! pip3 install -r requirements.txt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import random
import torch
from torch import nn, optim
import math
from IPython import display
import requests
import torch.nn.functional as F

### Pre-processing

Add the api calls to the csv

### Device

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Import CSV's

In [3]:
train_df = pd.read_csv('preprocessed_train.csv', sep=',', encoding='latin-1')
test_df = pd.read_csv("test.csv")

In [4]:
data = train_df[['keyword', 'location', 'sarcastic', 'not_sarcastic', 'negative', 'neutral', 'positive']]
target = train_df['target']
#print(X_data["neutral"])
train_data, val_data, train_target, val_target = train_test_split(data, target, test_size = 0.2)
print(train_target)
# split the dataset into training and validation sets
#train_data, val_data = 
#print(train_data[:]["sarcastic"])
#train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
#val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)


3027    1
4130    1
2593    0
6399    1
7446    0
       ..
578     1
5937    0
1500    1
4678    1
3906    0
Name: target, Length: 6088, dtype: int64


### Convert categorical features in a DataFrame to one-hot encoding

In [5]:
def convert_features_to_one_hot(df, feature_name_list):
  for feature_name in feature_name_list:
    df = pd.get_dummies(df, columns=[feature_name])
  
  return df
#Define the training set/test set from the imported data... x_train, x_val, etc... needs to be predefined

data_to_convert = ['train_data', 'val_data']

feature_list = ['location', 'keyword']
for i,ix in enumerate(data_to_convert):
  exec(f'{data_to_convert[i]} = convert_features_to_one_hot({ix}, {feature_list})')


In [6]:
print(len(train_data.keys()))
input_layers = len(train_data.keys())

2995


### Model draft 

In [7]:
class netmodel(nn.Module):
  def __init__(self, input_layer=1, num_hidden=1, node_per_hidden=32, droppout=0., LSTM_layers=0, outputs=2):
    super(netmodel, self).__init__()
    self.input_layer = input_layer
    self.num_hidden = num_hidden 
    self.node_per_hidden = node_per_hidden
    self.droppout = droppout 
    self.SLTM_layers = LSTM_layers 
    self.outputs = outputs 
    self.inputfc = nn.Linear(input_layer, node_per_hidden) 
    self.hiddenfc = [] 
    for i in range(num_hidden-1):
      self.hiddenfc.append(nn.Linear(node_per_hidden, node_per_hidden))
    self.lastfc = nn.Linear(node_per_hidden, outputs)

  def forward(self, x, debug=False):
    drop = nn.Dropout(p=self.droppout)
    #x = x.view(1,1)
    x = self.inputfc(x)
    x = F.relu(x)
    x = drop(x)
    for i in range(self.num_hidden-1):
      x = self.hiddenfc[i](x)
      x = F.relu(x)
      x = drop(x)
    
    x = self.lastfc(x)
    x = F.softmax(x, dim=1)
    return x 



In [8]:
Model = netmodel(input_layer=input_layers, num_hidden=3, droppout=0.3).to(device)

#Model.forward(torch.tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]))

In [15]:


batch_size = 32
print(len(train_data))
print(len(train_target))
#tensor_train = torch.utils.data.TensorDataset(train_data, train_target)
tensor_train = torch.tensor(train_data.values.astype(float), dtype=torch.float32).to(device)
tensor_train_labels = torch.tensor(train_target.values.astype(float), dtype=torch.long).to(device)
tensor_train_set = torch.utils.data.TensorDataset(tensor_train, tensor_train_labels)

# Convert validation data to PyTorch tensors and move to device
#tensor_val = torch.utils.data.TensorDataset(val_data, val_target)
tensor_val = torch.tensor(val_data.values.astype(float), dtype=torch.float32).to(device)
tensor_val_labels = torch.tensor(val_target.values.astype(float), dtype=torch.long).to(device)
tensor_val_set = torch.utils.data.TensorDataset(tensor_val, tensor_val_labels)

train_loader = torch.utils.data.DataLoader(tensor_train_set, batch_size=batch_size, shuffle=True)

val_loader = torch.utils.data.DataLoader(tensor_val_set, batch_size=batch_size, shuffle=True)

lr = 1e-3
lambda_l2 = 1e-3
loss_func = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(Model.parameters(), lr=lr, weight_decay=lambda_l2)



6088
6088


In [16]:
def train_model(epochs, model):
    model.train()
    for epoch in range(epochs):
      for batch_id, (data, label) in enumerate(train_loader):
        #print(batch_idx)


        pred = Model.forward(data)
        print(type(pred[0][0].item()))

        optimizer.zero_grad()
        loss = loss_func(pred, label)
        #print(loss)
        loss.backward()
        optimizer.step()
        if batch_id % 60 == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_id * len(data), len(train_loader.dataset),
            100. * batch_id / len(train_loader), loss.item()))

In [17]:
train_model(1, Model)

<class 'float'>
Train Epoch: 0 [0/6088 (0%)]	Loss: 0.695287
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'floa